In [1]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import evaluate
import numpy as np
from datasets import load_dataset

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

/home/rana/.cache/pypoetry/virtualenvs/zindi-z3yfXQo9-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,ignore_mismatched_sizes=True)

Some weights of MarianMTModel were not initialized from the model checkpoint at /home/rana/Projects/zindi/models/marian/marian_output/checkpoint-1700 and are newly initialized because the shapes did not match:
- model.decoder.layers.0.fc1.bias: found shape torch.Size([2048]) in the checkpoint and torch.Size([1024]) in the model instantiated
- model.decoder.layers.0.fc1.weight: found shape torch.Size([2048, 512]) in the checkpoint and torch.Size([1024, 512]) in the model instantiated
- model.decoder.layers.0.fc2.weight: found shape torch.Size([512, 2048]) in the checkpoint and torch.Size([512, 1024]) in the model instantiated
- model.decoder.layers.1.fc1.bias: found shape torch.Size([2048]) in the checkpoint and torch.Size([1024]) in the model instantiated
- model.decoder.layers.1.fc1.weight: found shape torch.Size([2048, 512]) in the checkpoint and torch.Size([1024, 512]) in the model instantiated
- model.decoder.layers.1.fc2.weight: found shape torch.Size([512, 2048]) in the checkpoin

In [1]:
model_name = '/home/rana/Projects/zindi/models/marian/marian_output/checkpoint-1700'

In [9]:
import tqdm as notebook_tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from sacrebleu.metrics import BLEU
from tqdm import tqdm

def translate(model, tokenizer, text, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=150).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

def calculate_bleu(references, hypotheses):
    bleu = BLEU()
    return bleu.corpus_score(hypotheses, [references]).score

def validate_model(model_name, dataset_name, split):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name,ignore_mismatched_sizes=True).to(device)

    # Load dataset
    dataset = load_dataset(dataset_name, split=split)

    # Translate and calculate BLEU score
    hypotheses = []
    references = []

    for batch in tqdm(dataset, desc="Translating"):
        source_text = batch['translation']['dyu']  # Adjust this based on your dataset's column names
        reference = batch['translation']['fr']  # Adjust this based on your dataset's column names

        # source_text.insert(0,"dyu")
        # source_text.append("<\s>")
        # source_text.append("fr")
        
        translation = translate(model, tokenizer, "dyu " + source_text + " <\s> fr ", device)[0]
        print(reference)
        print(translation)
        
        hypotheses.append(translation)
        references.append(reference)
        break

    bleu_score = calculate_bleu(references, hypotheses)
    print(f"BLEU Score: {bleu_score:.2f}")

# Run validation with GPU
# validate_model(model_checkpoint, "uvci/Koumankan_mt_dyu_fr", use_gpu=True)

# Run validation without GPU
validate_model(model_name, "uvci/Koumankan_mt_dyu_fr", "validation")

Using device: cpu


Some weights of MarianMTModel were not initialized from the model checkpoint at /home/rana/Projects/zindi/models/marian/marian_output/checkpoint-1700 and are newly initialized because the shapes did not match:
- model.decoder.layers.0.fc1.bias: found shape torch.Size([2048]) in the checkpoint and torch.Size([1024]) in the model instantiated
- model.decoder.layers.0.fc1.weight: found shape torch.Size([2048, 512]) in the checkpoint and torch.Size([1024, 512]) in the model instantiated
- model.decoder.layers.0.fc2.weight: found shape torch.Size([512, 2048]) in the checkpoint and torch.Size([512, 1024]) in the model instantiated
- model.decoder.layers.1.fc1.bias: found shape torch.Size([2048]) in the checkpoint and torch.Size([1024]) in the model instantiated
- model.decoder.layers.1.fc1.weight: found shape torch.Size([2048, 512]) in the checkpoint and torch.Size([1024, 512]) in the model instantiated
- model.decoder.layers.1.fc2.weight: found shape torch.Size([512, 2048]) in the checkpoin

Tu portes un nom de fantaisie.
si s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s 